In [1]:
# setup environment
!pip -q install gdown dash==2.* plotly pandas numpy

#import pacakegs
import os
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime
from dash import Dash, dcc, html, Input, Output

# Load data from Google Drive
FILE_ID = "1ZFsJv60oE0kAM-FgfvY4LXnFCaRIQfMn"
CSV_NAME = "nyc_crashes.csv"

def load_data() -> pd.DataFrame:
    if os.path.exists(CSV_NAME):
        return pd.read_csv(CSV_NAME, low_memory=False)

    import gdown
    url = f"https://drive.google.com/uc?id={FILE_ID}"
    gdown.download(url, CSV_NAME, quiet=False)
    return pd.read_csv(CSV_NAME, low_memory=False)

# Data cleaning/processing
def clean_and_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Drop irrelevant columns (safe if missing)
    df = df.drop(
        columns=["ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME", "LOCATION"],
        errors="ignore"
    )

    # Parse crash date/time
    if "CRASH DATE" in df.columns:
        df["CRASH DATE"] = pd.to_datetime(df["CRASH DATE"], errors="coerce")
    if "CRASH TIME" in df.columns:
        df["CRASH HOUR"] = pd.to_datetime(df["CRASH TIME"], format="%H:%M", errors="coerce").dt.hour

    # Time features
    df["YEAR"] = df["CRASH DATE"].dt.year if "CRASH DATE" in df.columns else np.nan
    df["MONTH"] = df["CRASH DATE"].dt.to_period("M").dt.to_timestamp() if "CRASH DATE" in df.columns else pd.NaT
    df["DAY_NAME"] = df["CRASH DATE"].dt.day_name() if "CRASH DATE" in df.columns else np.nan

    # Normalize text fields
    if "BOROUGH" in df.columns:
        df["BOROUGH"] = (
            df["BOROUGH"].astype(str).str.strip().str.upper()
            .replace({"NAN": np.nan, "NONE": np.nan, "": np.nan})
        )

    for col in ["VEHICLE TYPE CODE 1", "CONTRIBUTING FACTOR VEHICLE 1"]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.upper()
            df.loc[df[col].isin(["", "NAN", "NONE"]), col] = np.nan

    # Coordinates numeric + remove (0,0)
    for col in ["LATITUDE", "LONGITUDE"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    if "LATITUDE" in df.columns and "LONGITUDE" in df.columns:
        df = df.dropna(subset=["LATITUDE", "LONGITUDE"])
        df = df[(df["LATITUDE"] != 0) & (df["LONGITUDE"] != 0)]

    # Injuries/fatalities numeric
    for col in ["NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

    return df

df_raw = load_data()
df = clean_and_features(df_raw)

five_boroughs = ["BRONX", "BROOKLYN", "MANHATTAN", "QUEENS", "STATEN ISLAND"]

all_years = sorted(df["YEAR"].dropna().unique().astype(int)) if "YEAR" in df.columns else []
min_year = int(min(all_years)) if all_years else 2012
max_year = int(max(all_years)) if all_years else 2024

borough_options = [{"label": b.title(), "value": b} for b in sorted(df["BOROUGH"].dropna().unique())] if "BOROUGH" in df.columns else []
vehicle_options = [{"label": v.title(), "value": v} for v in sorted(df["VEHICLE TYPE CODE 1"].dropna().unique())] if "VEHICLE TYPE CODE 1" in df.columns else []

MAP_STYLE = "open-street-map"

# Dash setup
app = Dash(__name__)
app.title = "NYC Collisions Dashboard"

def kpi_card(title: str, value: str) -> html.Div:
    return html.Div(
        style={
            "border": "1px solid #ddd",
            "borderRadius": "10px",
            "padding": "12px 14px",
            "minWidth": "220px",
            "backgroundColor": "white",
        },
        children=[
            html.Div(title, style={"fontSize": "12px", "color": "#555", "marginBottom": "4px"}),
            html.Div(value, style={"fontSize": "22px", "fontWeight": "600"}),
        ],
    )

app.layout = html.Div(
    style={"fontFamily": "Arial", "backgroundColor": "#f6f6f6", "padding": "14px"},
    children=[
        html.Div(
            style={"maxWidth": "1200px", "margin": "0 auto"},
            children=[
                html.H1("NYC Motor Vehicle Collisions Dashboard", style={"margin": "0 0 6px 0"}),
                html.Div("Created by Durim Berisha, Riley Weaver, Emily Moran-Perez and Ryan Rahman (Group #5).", style={"color": "#444"}),

                # Filters
                html.Div(
                    style={
                        "display": "flex",
                        "gap": "14px",
                        "flexWrap": "wrap",
                        "marginTop": "12px",
                        "padding": "12px",
                        "border": "1px solid #ddd",
                        "borderRadius": "12px",
                        "backgroundColor": "white",
                    },
                    children=[
                        html.Div(
                            style={"minWidth": "280px", "flex": "1"},
                            children=[
                                html.Div("Year range", style={"fontSize": "12px", "color": "#555", "marginBottom": "6px"}),
                                dcc.RangeSlider(
                                    id="year_range",
                                    min=min_year,
                                    max=max_year,
                                    step=1,
                                    value=[min_year, max_year],
                                    marks={y: str(y) for y in range(min_year, max_year + 1, max(1, (max_year - min_year)//6 or 1))},
                                    allowCross=False,
                                ),
                            ],
                        ),
                        html.Div(
                            style={"minWidth": "320px", "flex": "1"},
                            children=[
                                html.Div("Borough", style={"fontSize": "12px", "color": "#555", "marginBottom": "6px"}),
                                dcc.Dropdown(
                                    id="boroughs",
                                    options=borough_options,
                                    value=five_boroughs,
                                    multi=True,
                                    placeholder="Select borough(s)",
                                ),
                            ],
                        ),
                        html.Div(
                            style={"minWidth": "340px", "flex": "1"},
                            children=[
                                html.Div("Vehicle Type (optional)", style={"fontSize": "12px", "color": "#555", "marginBottom": "6px"}),
                                dcc.Dropdown(
                                    id="vehicle_type",
                                    options=vehicle_options,
                                    value=None,
                                    multi=False,
                                    placeholder="Filter to a vehicle type",
                                    clearable=True,
                                ),
                            ],
                        ),
                    ],
                ),

                html.Div(
                    id="kpi_row",
                    style={
                        "display": "grid",
                        "gridTemplateColumns": "repeat(auto-fit, minmax(220px, 1fr))",
                        "gap": "12px",
                        "marginTop": "12px",
                    },
                ),

                # Heatmap is now the 3rd tab
                dcc.Tabs(
                    id="tabs",
                    value="tab_overview",
                    style={"marginTop": "12px"},
                    children=[
                        dcc.Tab(label="Overview", value="tab_overview"),
                        dcc.Tab(label="Time Trends", value="tab_time"),
                        dcc.Tab(label="Heatmap", value="tab_heatmap"),
                        dcc.Tab(label="Map", value="tab_map"),
                        dcc.Tab(label="Vehicle + Borough", value="tab_vehicle_borough"),
                    ],
                ),

                html.Div(id="tab_content", style={"marginTop": "12px"}),
            ],
        )
    ],
)

# helpers
def apply_filters(dff: pd.DataFrame, year_min: int, year_max: int, boroughs, vehicle_type):
    dff = dff.copy()

    if "YEAR" in dff.columns:
        dff = dff[(dff["YEAR"] >= year_min) & (dff["YEAR"] <= year_max)]

    if boroughs and "BOROUGH" in dff.columns:
        dff = dff[dff["BOROUGH"].isin(boroughs)]

    if vehicle_type and "VEHICLE TYPE CODE 1" in dff.columns:
        dff = dff[dff["VEHICLE TYPE CODE 1"] == vehicle_type]

    return dff

def empty_fig(title: str):
    fig = px.scatter(title=title)
    fig.update_layout(
        annotations=[dict(text="No data for selected filters", x=0.5, y=0.5, showarrow=False, xref="paper", yref="paper")]
    )
    return fig

def fig_monthly_line(dff: pd.DataFrame):
    if "MONTH" not in dff.columns or "COLLISION_ID" not in dff.columns:
        return empty_fig("Crashes Over Time (Monthly)")

    month_counts = (
        dff.dropna(subset=["MONTH"])
           .groupby("MONTH", as_index=False)["COLLISION_ID"]
           .count()
           .rename(columns={"COLLISION_ID": "CRASHES"})
           .sort_values("MONTH")
    )

    if len(month_counts) == 0:
        return empty_fig("Crashes Over Time (Monthly)")

    fig = px.line(month_counts, x="MONTH", y="CRASHES", title="Crashes Over Time (Monthly)")
    fig.update_xaxes(type="date", tickangle=30)

    y_max = float(month_counts["CRASHES"].max()) if len(month_counts) else 0

    lockdown_date = datetime(2020, 4, 1)
    fig.add_vline(x=lockdown_date, line_width=2, line_dash="dash")
    fig.add_annotation(
        x=lockdown_date,
        y=y_max,
        text="COVID-19 lockdown (04/2020)",
        showarrow=True,
        yanchor="bottom",
    )
    return fig

def fig_top_factors(dff: pd.DataFrame):
    col = "CONTRIBUTING FACTOR VEHICLE 1"
    if col not in dff.columns:
        return empty_fig("Top 15 Contributing Factors (Vehicle 1)")

    factor_counts = (
        dff.dropna(subset=[col])[col]
          .value_counts()
          .head(15)
          .reset_index(name="CRASHES")
          .rename(columns={"index": col})
    )

    if len(factor_counts) == 0:
        return empty_fig("Top 15 Contributing Factors (Vehicle 1)")

    fig = px.bar(
        factor_counts,
        x=col,
        y="CRASHES",
        title="Top 15 Contributing Factors (Vehicle 1)"
    )
    fig.update_xaxes(tickangle=25)
    return fig

def fig_vehicle_bubbles(dff: pd.DataFrame):
    col = "VEHICLE TYPE CODE 1"
    if "YEAR" not in dff.columns or col not in dff.columns or "COLLISION_ID" not in dff.columns:
        return empty_fig("Gapminder-style: Vehicle Type Over Time")

    tmp = dff.dropna(subset=["YEAR", col]).copy()
    if len(tmp) == 0:
        return empty_fig("Gapminder-style: Vehicle Type Over Time")

    top_types = tmp[col].value_counts().head(12).index
    tmp = tmp[tmp[col].isin(top_types)]

    agg = tmp.groupby(["YEAR", col], as_index=False).agg(
        crashes=("COLLISION_ID", "count"),
        injured=("NUMBER OF PERSONS INJURED", "sum"),
        killed=("NUMBER OF PERSONS KILLED", "sum"),
    )
    agg["killed_size"] = agg["killed"].clip(lower=1)

    fig = px.scatter(
        agg,
        x="injured",
        y="crashes",
        size="killed_size",
        color=col,
        animation_frame="YEAR",
        animation_group=col,
        size_max=60,
        title="Gapminder-style: Vehicle Type Over Time",
        labels={"injured": "Total Injured", "crashes": "Total Crashes"},
    )
    return fig

def fig_borough_bubbles(dff: pd.DataFrame):
    if "YEAR" not in dff.columns or "BOROUGH" not in dff.columns or "COLLISION_ID" not in dff.columns:
        return empty_fig("Gapminder-style: Borough Trends Over Time")

    tmp = dff.dropna(subset=["YEAR", "BOROUGH"]).copy()
    tmp = tmp[tmp["BOROUGH"].isin(five_boroughs)]

    if len(tmp) == 0:
        return empty_fig("Gapminder-style: Borough Trends Over Time")

    agg = tmp.groupby(["YEAR", "BOROUGH"], as_index=False).agg(
        crashes=("COLLISION_ID", "count"),
        injured=("NUMBER OF PERSONS INJURED", "sum"),
        killed=("NUMBER OF PERSONS KILLED", "sum"),
    )
    agg["killed_size"] = agg["killed"].clip(lower=1)

    fig = px.scatter(
        agg,
        x="injured",
        y="crashes",
        size="killed_size",
        color="BOROUGH",
        animation_frame="YEAR",
        animation_group="BOROUGH",
        text="BOROUGH",
        title="Gapminder-style: Borough Trends Over Time",
        size_max=70,
        category_orders={"BOROUGH": five_boroughs},
    )
    fig.update_traces(textposition="top center", cliponaxis=False, opacity=0.7)
    fig.update_layout(margin=dict(l=70, r=170, t=90, b=70))
    return fig

def fig_map(dff: pd.DataFrame):
    if "LATITUDE" not in dff.columns or "LONGITUDE" not in dff.columns:
        return empty_fig("NYC Crash Locations by Borough (sampled for speed)")

    geo = dff.dropna(subset=["LATITUDE", "LONGITUDE"]).copy()
    if "BOROUGH" in geo.columns:
        geo["BOROUGH"] = geo["BOROUGH"].fillna("UNKNOWN").str.title().str.strip()
    else:
        geo["BOROUGH"] = "Unknown"

    geo = geo[
        (geo["LATITUDE"].between(40.45, 40.95)) &
        (geo["LONGITUDE"].between(-74.30, -73.65))
    ]

    if len(geo) == 0:
        return empty_fig("NYC Crash Locations by Borough (sampled for speed)")

    geo = (
        geo.groupby("BOROUGH", group_keys=False)
           .apply(lambda x: x.sample(min(len(x), 5000), random_state=42))
           .reset_index(drop=True)
    )

    fig = px.scatter_mapbox(
        geo,
        lat="LATITUDE",
        lon="LONGITUDE",
        color="BOROUGH",
        hover_data=["CRASH DATE", "NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED"] if "CRASH DATE" in geo.columns else ["NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED"],
        zoom=9.5,
        title="NYC Crash Locations by Borough (sampled for speed)",
        mapbox_style=MAP_STYLE,
    )
    fig.update_traces(marker=dict(opacity=0.55))
    fig.update_layout(margin=dict(l=0, r=0, t=55, b=0))
    return fig

# heatmap day vs hour crash (to find out if weekend is safer than weekdays)
def fig_day_hour_heatmap(dff: pd.DataFrame):
    if "DAY_NAME" not in dff.columns or "CRASH HOUR" not in dff.columns or "COLLISION_ID" not in dff.columns:
        return empty_fig("Crash Frequency by Day and Hour")

    day_order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    tmp = dff.dropna(subset=["DAY_NAME","CRASH HOUR"]).copy()

    if len(tmp) == 0:
        return empty_fig("Crash Frequency by Day and Hour")

    tmp["DAY_NAME"] = pd.Categorical(tmp["DAY_NAME"], categories=day_order, ordered=True)
    pivot = tmp.pivot_table(index="DAY_NAME", columns="CRASH HOUR", values="COLLISION_ID", aggfunc="count").fillna(0)

    fig = px.imshow(pivot, title="Crash Frequency by Day and Hour")
    fig.update_layout(margin=dict(l=70, r=30, t=60, b=50))
    return fig

# callbacks
@app.callback(
    Output("kpi_row", "children"),
    Input("year_range", "value"),
    Input("boroughs", "value"),
    Input("vehicle_type", "value"),
)
def update_kpis(year_range, boroughs, vehicle_type):
    y0, y1 = year_range
    dff = apply_filters(df, y0, y1, boroughs, vehicle_type)

    total_crashes = len(dff)
    total_injured = int(dff["NUMBER OF PERSONS INJURED"].sum()) if "NUMBER OF PERSONS INJURED" in dff.columns else 0
    total_killed = int(dff["NUMBER OF PERSONS KILLED"].sum()) if "NUMBER OF PERSONS KILLED" in dff.columns else 0

    return [
        kpi_card("Total crashes", f"{total_crashes:,}"),
        kpi_card("Total injured", f"{total_injured:,}"),
        kpi_card("Total killed", f"{total_killed:,}"),
    ]

@app.callback(
    Output("tab_content", "children"),
    Input("tabs", "value"),
    Input("year_range", "value"),
    Input("boroughs", "value"),
    Input("vehicle_type", "value"),
)
def render_tab(tab, year_range, boroughs, vehicle_type):
    y0, y1 = year_range
    dff = apply_filters(df, y0, y1, boroughs, vehicle_type)

    # Overview: ALL visuals
    if tab == "tab_overview":
        return html.Div(
            style={
                "display": "grid",
                "gridTemplateColumns": "repeat(auto-fit, minmax(420px, 1fr))",
                "gap": "12px"
            },
            children=[
                html.Div(
                    style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
                    children=[dcc.Graph(figure=fig_monthly_line(dff), config={"responsive": True}, style={"height": "62vh"})],
                ),
                html.Div(
                    style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
                    children=[dcc.Graph(figure=fig_top_factors(dff), config={"responsive": True}, style={"height": "62vh"})],
                ),
                html.Div(
                    style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
                    children=[dcc.Graph(figure=fig_day_hour_heatmap(dff), config={"responsive": True}, style={"height": "62vh"})],
                ),
                html.Div(
                    style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
                    children=[dcc.Graph(figure=fig_map(dff), config={"responsive": True}, style={"height": "62vh"})],
                ),
                html.Div(
                    style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
                    children=[dcc.Graph(figure=fig_vehicle_bubbles(dff), config={"responsive": True}, style={"height": "62vh"})],
                ),
                html.Div(
                    style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
                    children=[dcc.Graph(figure=fig_borough_bubbles(dff), config={"responsive": True}, style={"height": "62vh"})],
                ),
            ],
        )

    # Time Trends: crash over time + factors
    if tab == "tab_time":
        return html.Div(
            style={
                "display": "grid",
                "gridTemplateColumns": "repeat(auto-fit, minmax(420px, 1fr))",
                "gap": "12px"
            },
            children=[
                html.Div(
                    style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
                    children=[dcc.Graph(figure=fig_monthly_line(dff), config={"responsive": True}, style={"height": "70vh"})],
                ),
                html.Div(
                    style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
                    children=[dcc.Graph(figure=fig_top_factors(dff), config={"responsive": True}, style={"height": "70vh"})],
                ),
            ],
        )

    # Heatmap: day vs hour crash (3rd tab)
    if tab == "tab_heatmap":
        return html.Div(
            style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
            children=[dcc.Graph(figure=fig_day_hour_heatmap(dff), config={"responsive": True}, style={"height": "80vh"})],
        )

    # Map: map
    if tab == "tab_map":
        return html.Div(
            style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
            children=[dcc.Graph(figure=fig_map(dff), config={"responsive": True}, style={"height": "80vh"})],
        )

    # Last tab: vehicle + borough
    if tab == "tab_vehicle_borough":
        return html.Div(
            style={
                "display": "grid",
                "gridTemplateColumns": "repeat(auto-fit, minmax(420px, 1fr))",
                "gap": "12px"
            },
            children=[
                html.Div(
                    style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
                    children=[dcc.Graph(figure=fig_vehicle_bubbles(dff), config={"responsive": True}, style={"height": "70vh"})],
                ),
                html.Div(
                    style={"backgroundColor": "white", "border": "1px solid #ddd", "borderRadius": "12px", "padding": "10px"},
                    children=[dcc.Graph(figure=fig_borough_bubbles(dff), config={"responsive": True}, style={"height": "70vh"})],
                ),
            ],
        )

    return html.Div("Tab not found.")

# run (Colab)
#app.run(jupyter_mode="inline", debug=True)

# run and click the link for external browser to open
app.run(jupyter_mode="external", debug=True, port=8051)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 9.6 MB/s eta 0:00:00


Downloading...
From (original): https://drive.google.com/uc?id=1ZFsJv60oE0kAM-FgfvY4LXnFCaRIQfMn
From (redirected): https://drive.google.com/uc?id=1ZFsJv60oE0kAM-FgfvY4LXnFCaRIQfMn&confirm=t&uuid=a741e21f-f0d3-4716-bc4e-92fe6a37b233
To: /content/nyc_crashes.csv
100%|██████████| 211M/211M [00:01<00:00, 115MB/s]


Dash app running on:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>